# Computing the minimal supercell size to host the muon: the `IsolatedImpurityWorkChain`

The minimal size of the supercell that allows the convergence of the calculations is a crucial parameter that must be automatically determined in this workflow.

Starting from the input structure, the first step (a) consists of generating a nearly cubic supercell (implemented using the `CubicSupercellTransformation` Python class of the `Pymatgen` package. The size of the generated supercell is controlled by two parameters:  a minimum length for the supercell and a minimum number of atoms allowed.  These are optional input parameters, whose default values are $l_{at}+1~\text{\AA}$ and $N_{at}+1$, respectively, where $l_{at}$ is the length of the smallest lattice vector of the input structure and $N_{at}$ is the number of atoms in the input structure. Step (b) is accomplished by selecting one Voronoi interstitial node in the unit cell by means of the `VoronoiInterstitialGenerator` Python class of the `Pymatgen` package, and inserting an atomic site at that position in the supercell using a hydrogen pseudopotential, in order to mimic the muon. The forces acting on all atoms are then obtained from a converged self-consistent DFT calculation in step (c) and are used to check for convergence in step (d). 

The forces obtained with a single SCF calculation decay exponentially with their distance from the muon. The decay length $\lambda$ is obtained as the best fit to $F \exp(-\lambda r_{i})$. Notice that an unrelaxed charged supercell, even without the muon, can show forces on the host atoms. For this reason we always consider the difference between the force on each atom with and without the muon (in the uncharged case the latter vanish).

We assume that convergence is reached when atomic forces decay below a given threshold $\Delta F$, which in the workflow is an optional input parameter, with the default set to $1 \times 10^{-3}$ Ry/Bohr or 0.0257 eV/$\AA$. To obtain a converged supercell, two conditions that ensure vanishing forces within the cell have to be satisfied: the minimum host atomic force is less than $\Delta F$ and the maximum $r_\mathrm{{i}}$  distance is greater than the minimum convergence distance, $\ln\frac{\Delta F}{F}/(-\lambda)$. If convergence is achieved, the workflow returns the supercell used in the last step and the corresponding transformation matrix with respect to the input structure. If convergence is not achieved, a larger supercell is generated and the loop goes back to step (c), provided that the maximum number of iteration is not exceeded.

<img src="source/images/supercell_convergence.png" width="400">


## How to prepare and run a simple `IsolatedImpurityWorkChain`

In [2]:
# Loading the aiida profile.
from aiida import load_profile, orm
load_profile()

# Loading other additional aiida modules.
from aiida.engine import submit, run_get_node
from aiida.plugins import DataFactory

# Loading the pymatgen CifParser module; not mandatory, we use it here as we have the structure saved in a cif file.
from pymatgen.io.cif import CifParser

# Loading the WorkChain.
from aiida_impuritysupercellconv.workflows.impuritysupercellconv import IsolatedImpurityWorkChain

### Setting the data

Here below we parse the structure, the magnetic moments, and we set some other relevant inputs for the workflow:

In [6]:
# StructureData and magmom.
parser = CifParser("../examples/Si.cif")
py_struc = parser.get_structures(primitive=False)[0]
aiida_structure = orm.StructureData(pymatgen=py_struc)

# Codes.
codename = "pw-7.2@localhost"  # edit 
code = orm.load_code(codename)

# Resources dictionary with a minimal computational settings.
resources = {
    "num_machines": 1,
    "num_mpiprocs_per_machine": 8,
}

### (1) Getting the builder from *get_builder_from_protocol()* function.

In [7]:
builder = IsolatedImpurityWorkChain.get_builder_from_protocol(
    pw_code = code,
    structure = aiida_structure,
    relax_unitcell = False,
)

builder.pwscf.kpoints_distance = orm.Float(0.4)

builder.pwscf.pw.metadata.options.resources = resources
builder.pwscf.pw.metadata.options.queue_name = "daily"
builder.pwscf.pw.metadata.description = "Testing the MuusConvWorkChain via protocols"
builder.pwscf.pw.metadata.options.prepend_text = "export OMP_NUM_THREADS=1"

if 'relax' in builder: # i.e. relax_unitcell = True
    builder.relax.base.pw.metadata.options.resources = resources

submit the workchain, just run:

```python
submission = run_get_node(builder)
```